In [ ]:
#!/usr/local/bin/python3.6

import numpy as np
import matplotlib.pyplot as plt
import os
import hashlib

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from scapy.all import *
from keras.preprocessing.text import hashing_trick
from keras.preprocessing.text import text_to_word_sequence


packets_testing = rdpcap('./pcap/feeding.pcapng')
packets_feeding = rdpcap('./pcap/testing.pcapng')

In [ ]:
#traitement des paquets ###############################################

#feeding tab ##########################################################

i = 0
feeding_tab = []
for y in range(len(packets_feeding)+1):
    feeding_tab.append(["" for y in range(25)])


for packet in packets_feeding:
    ###[ Ethernet ]###
    feeding_tab[i][0] = packet[0].dst
    feeding_tab[i][1] = packet[0].src
    feeding_tab[i][2] = packet[0].type
    ###[ IP ]###
    feeding_tab[i][3] = packet[1].version
    feeding_tab[i][4] = packet[1].ihl
    feeding_tab[i][5] = packet[1].tos
    feeding_tab[i][6] = packet[1].len
    feeding_tab[i][7] = packet[1].id
    feeding_tab[i][8] = packet[1].frag
    feeding_tab[i][9] = packet[1].ttl
    feeding_tab[i][10] = packet[1].proto
    feeding_tab[i][11] = packet[1].chksum
    feeding_tab[i][12] = packet[1].src
    feeding_tab[i][13] = packet[1].dst
    ###[ TCP ]###
    feeding_tab[i][14] = packet[2].sport
    feeding_tab[i][15] = packet[2].dport
    feeding_tab[i][16] = packet[2].seq
    feeding_tab[i][17] = packet[2].ack
    feeding_tab[i][18] = packet[2].dataofs
    feeding_tab[i][19] = packet[2].reserved
    feeding_tab[i][20] = packet[2].window
    feeding_tab[i][21] = packet[2].chksum
    feeding_tab[i][22] = packet[2].urgptr
    if packet[2].options is not None:
        feeding_tab[i][23] = packet[2].options
    else:
        feeding_tab[i][23] = '0'
    ###[ Raw ]###
    if "Raw" in packet:
        feeding_tab[i][24] = packet[3].load
    else:
        feeding_tab[i][24] = '0'

    #print(feeding_tab[i][] = [26])
    i += 1

In [ ]:
#testing tab ##########################################################

i = 0
testing_tab = []
for y in range(len(packets_testing)+1):
    testing_tab.append(["" for y in range(25)])


for packet in packets_testing:
    #packet.show()
    ###[ Ethernet ]###
    testing_tab[i][0] = packet[0].dst
    testing_tab[i][1] = packet[0].src
    testing_tab[i][2] = packet[0].type
    ###[ IP ]###
    testing_tab[i][3] = packet[1].version
    testing_tab[i][4] = packet[1].ihl
    testing_tab[i][5] = packet[1].tos
    testing_tab[i][6] = packet[1].len
    testing_tab[i][7] = packet[1].id
    testing_tab[i][8] = packet[1].frag
    testing_tab[i][9] = packet[1].ttl
    testing_tab[i][10] = packet[1].proto
    testing_tab[i][11] = packet[1].chksum
    testing_tab[i][12] = packet[1].src
    testing_tab[i][13] = packet[1].dst
    ###[ TCP ]###
    testing_tab[i][14] = packet[2].sport
    testing_tab[i][15] = packet[2].dport
    testing_tab[i][16] = packet[2].seq
    testing_tab[i][17] = packet[2].ack
    testing_tab[i][18] = packet[2].dataofs
    testing_tab[i][19] = packet[2].reserved
    testing_tab[i][20] = packet[2].window
    testing_tab[i][21] = packet[2].chksum
    testing_tab[i][22] = packet[2].urgptr
    if packet[2].options is not None:
        testing_tab[i][23] = packet[2].options
    else:
        testing_tab[i][23] = ""
    ###[ Raw ]###
    if "Raw" in packet:
        testing_tab[i][24] = packet[3].load
    else:
        testing_tab[i][24] = ""

    #print(testing_tab[i][] = [26])
    i += 1

In [ ]:
count = 0
menace = 0
feeding_menace = []
for x in range(len(feeding_tab)):
    feeding_menace.append(0)

for packet in feeding_tab:
    if re.search('\.\.\/', str(packet)):
        feeding_menace[count] = 1
        count += 1
    elif re.search('Nikto', str(packet)):
        feeding_menace[count] = 1
        count += 1
    elif re.search('cmd=', str(packet)):
        feeding_menace[count] = 1
        count += 1
    elif re.search('%2F', str(packet)):
        feeding_menace[count] = 1
        count += 1
    elif re.search('passwd', str(packet)):
        feeding_menace[count] = 1
        count += 1
    else:
        feeding_menace[count] = 0
        count += 1
        menace += 1
        
print(menace)
print(count)

In [ ]:
count = 0
menace = 0
testing_menace = []
for x in range(len(testing_tab)):
    testing_menace.append(0)

for packet in testing_tab:
    if re.search('\.\.\/', str(packet)):
        testing_menace[count] = 1
        count += 1
    elif re.search('Nikto', str(packet)):
        testing_menace[count] = 1
        count += 1
    elif re.search('cmd=', str(packet)):
        testing_menace[count] = 1
        count += 1
    elif re.search('%2F', str(packet)):
        testing_menace[count] = 1
        count += 1
    elif re.search('passwd', str(packet)):
        testing_menace[count] = 1
        count += 1
    else:
        testing_menace[count] = 0
        count += 1
        menace += 1
        
print(menace)
print(count)

In [ ]:
# tokenization des donnees #########################


def computeMD5hash(my_string):
    m = hashlib.md5()
    m.update(my_string.encode('utf-8'))
    digest = m.hexdigest()
    number = int(digest, 16)
    return number




for i in range(len(feeding_tab)):
    for j in range(25):
        text = feeding_tab[i][j]
        if str(text) == '' or str(0) == str(text):
            text = 0
        if type(text) == bytes:
            if re.match(b'\x00',text):
                text = 0
        if type(text) == str:
            words = set(text_to_word_sequence(text))
            vocab_size = len(words)
            tmp = hashing_trick(text, round(vocab_size*1.3), hash_function='md5')
            check = ''.join(map(str, tmp))
            if type(check) != int:
                feeding_tab[i][j] = computeMD5hash(str(check))
            else:
                feeding_tab[i][j] = computeMD5hash(str(check))



        elif type(text) == list or type(text) == bytes:
            text = ' '.join(map(str, feeding_tab[i][j]))
            words = set(text_to_word_sequence(text))
            vocab_size = len(words)
            tmp = hashing_trick(text, round(vocab_size*1.3), hash_function='md5')
            check = ''.join(map(str, tmp))
            if type(check) != int:
                feeding_tab[i][j] = computeMD5hash(str(check))
            else:
                feeding_tab[i][j] = computeMD5hash(str(check))


        elif type(text) == int:
            feeding_tab[i][j] = text

        print(feeding_tab[i][j])
        

#################################################################
#################################################################


for i in range(len(testing_tab)):
    for j in range(25):
        text = testing_tab[i][j]
        if str(text) == '' or str(0) == str(text):
            text = 0
        if type(text) == bytes:
            if re.match(b'\x00',text):
                text = 0
        if type(text) == str:
            words = set(text_to_word_sequence(text))
            vocab_size = len(words)
            tmp = hashing_trick(text, round(vocab_size*1.3), hash_function='md5')
            check = ''.join(map(str, tmp))
            if type(check) != int:
                testing_tab[i][j] = computeMD5hash(str(check))
            else:
                testing_tab[i][j] = computeMD5hash(str(check))



        elif type(text) == list or type(text) == bytes:
            text = ' '.join(map(str, testing_tab[i][j]))
            words = set(text_to_word_sequence(text))
            vocab_size = len(words)
            tmp = hashing_trick(text, round(vocab_size*1.3), hash_function='md5')
            check = ''.join(map(str, tmp))
            if type(check) != int:
                testing_tab[i][j] = computeMD5hash(str(check))
            else:
                testing_tab[i][j] = computeMD5hash(str(check))


        elif type(text) == int:
            testing_tab[i][j] = text

        print(testing_tab[i][j])

In [ ]:
# reshape éventuel ###############################################

##################################################################

nb_classes = 2

X_train = np.array(feeding_tab)
Y_train = np.array(feeding_menace )
X_test = np.array(testing_tab)
Y_test = np.array(testing_menace )


Y_train = np_utils.to_categorical(Y_train, nb_classes)
Y_test = np_utils.to_categorical(Y_test, nb_classes)

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)
'''



X_test = np.array(X_test)


Y_test = np.array(Y_test)

Y_test = np_utils.to_categorical(Y_test, nb_classes)


#X_train = np.reshape(X_train, (8208,)).T
#X_test =  np.reshape(X_test, (30560,)).T





print(X_test.shape)
print(Y_test.shape)


(60000, 784)
(60000, 10)
(10000, 784)
(10000, 10)
'''



In [ ]:
# création du modèle

model = Sequential()
model.add(Dense(2, input_shape=(25,)))
model.add(Activation('relu')) # An "activation" is just a non-linear function applied to the output
                              # of the layer above. Here, with a "rectified linear unit",
                              # we clamp all values below 0 to 0.
                           
model.add(Dropout(0.2))   # Dropout helps protect the model from memorizing or "overfitting" the training data
model.add(Dense(2))
model.add(Dropout(0.2))
model.add(Activation('softmax')) # This special "softmax" activation among other things,
                                 # ensures the output is a valid probaility distribution, that is
                                 # that its values are all non-negative and sum to 1.


In [ ]:
# compilation du model ###############################################

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# training  ###############################################

model.fit(X_train, Y_train, batch_size=128, epochs=4, verbose=1,
         validation_data=(X_test, Y_test))

#model.fit(X_train, Y_train, batch_size=128, nb_epoch=4, verbose=1,
         #validation_data=(X_test, Y_test))

In [ ]:
  # evualtion ###############################################
         
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# résultat ###############################################

# The predict_classes function outputs the highest probability class
# according to the trained classifier for each input example.
predicted_classes = model.predict_classes(X_test)
for i in predicted_classes:
    print(i)
    print(testing_tab[i])
# Check which items we got right / wrong
correct_indices = np.nonzero(predicted_classes == testing_menace)[0]
incorrect_indices = np.nonzero(predicted_classes != testing_menace)[0]

In [ ]:
print(correct_indices)
print(incorrect_indices)